In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 3

In [ ]:
sig = pd.read_excel(
    'gmo_analysis_data.xlsx',
    sheet_name='signals',
    index_col=0,
    parse_dates=True
)

tr = pd.read_excel(
    'gmo_analysis_data.xlsx',
    sheet_name='total returns',
    index_col=0,
    parse_dates=True
)

df = sig.join(tr['SPY'])
df['dp_lag'] = df['SPX D/P'].shift(1)
df['ep_lag'] = df['SPX E/P'].shift(1)
df['t10_lag'] = df['T-Note 10YR'].shift(1)
df = df.dropna()

y = df['SPY']

# 3.1

# Only DP
x1 = sm.add_constant(df['dp_lag'])
m1 = sm.OLS(y, x1).fit()
print('DP only R2:', m1.rsquared)
print('alpha, beta:', m1.params.values)

# Only EP
x2 = sm.add_constant(df['ep_lag'])
m2 = sm.OLS(y, x2).fit()
print('EP only R2:', m2.rsquared)
print('alpha, beta:', m2.params.values)

# DP EP T10
x3 = sm.add_constant(df[['dp_lag','ep_lag','t10_lag']])
m3 = sm.OLS(y, x3).fit()
print('DP EP T10 R2:', m3.rsquared)
print('alpha, betas:', m3.params.values)

# 3.2
pred1 = m1.predict(x1)
pred2 = m2.predict(x2)
pred3 = m3.predict(x3)

w1 = 100 * pred1
w2 = 100 * pred2
w3 = 100 * pred3

r1 = w1 * df['SPY']
r2 = w2 * df['SPY']
r3 = w3 * df['SPY']

def performance(r):
    mu = r.mean()
    voli = r.std(ddof=0)
    sr = mu / voli

    wealth_index = 1000 * (1 + r).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    maxdd = drawdowns.min()

    q5 = r.quantile(0.05)

    mdl = sm.OLS(r, sm.add_constant(df['SPY'])).fit()
    alpha = mdl.params['const']
    beta = mdl.params['SPY']
    te = mdl.resid.std(ddof=0) * np.sqrt(12)
    info = alpha * 12 / te

    return mu, voli, sr, maxdd, q5, alpha, beta, info

result = pd.DataFrame(
    [performance(r1), performance(r2), performance(r3)],
    index=['DP', 'EP', 'DP_EP_T10'],
    columns=['Mean','Vol','SR','Maxdd','q5','alpha','beta','IR']
)

print('\n')
print(result)

DP only R2: 0.008635874130828225
alpha, beta: [-0.0097799   1.02429981]
EP only R2: 0.0031842229435176117
alpha, beta: [-0.00285638  0.22199378]
DP EP T10 R2: 0.010242493642037331
alpha, betas: [-0.00361391  1.48450938 -0.2392071  -0.05721477]


               Mean       Vol        SR     Maxdd        q5     alpha  \
DP         0.009259  0.049209  0.188157 -0.710906 -0.060589  0.000984   
EP         0.008182  0.042686  0.191667 -0.612060 -0.062853  0.000278   
DP_EP_T10  0.009577  0.049346  0.194073 -0.681156 -0.060945  0.001363   

               beta        IR  
DP         0.952217  0.135876  
EP         0.909506  0.070317  
DP_EP_T10  0.945151  0.182506  


3.3 According to the GMO case, U.S. stocks underperformed short-term bonds over 2000–2011, which implies that holding only the risk-free asset would have returned roughly 0.15% per month. The three dynamic strategies generated average monthly returns of 0.93%, 0.82% and 0.96%. Since each of these clearly exceeds the roughly 0.15% monthly risk-free return, none of the strategies under-performed the risk-free rate over this period.


In [ ]:
neg_dp = (pred1 < 0).sum()
neg_ep = (pred2 < 0).sum()
neg_dp_ep_t10 = (pred3 < 0).sum()

print(neg_dp, neg_ep, neg_dp_ep_t10)
print(pred1.min(), pred2.min(), pred3.min())

0 0 1
0.0010315881386355231 0.0038935265463603506 -0.00031340065585346195


3.4 Neither the DP or the EP model ever forecasts a negative excess return. The combined DP-EP-T10 model does so in one period.


3.5 The dynamic strategies do not takes on more risk than the market. DP’s monthly volatility is 4.92%, EP’s is 4.27% and the combined model’s is 4.93%, while SPY’s is 5.31%. Their betas sit below one at 0.952, 0.910 and 0.945. Maximum drawdowns of –71.1%, –61.2% and –68.1% reflect timing swings rather than higher sustained exposure. Each strategy generated average monthly returns of 0.93%, 0.82% and 0.96% and delivered Sharpe ratios around 0.19. These results show the extra return comes from timing skill rather than simply taking on more risks.


Reference:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html
